In [32]:
%%bash
pip install xgboost

In [33]:
import pandas as pd

technical_indicators_binary = pd.read_csv('/home/jovyan/technical_indicators_binary.csv', index_col=0)

In [34]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost')


import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, cross_val_predict

# Assuming 'technical_indicators_binary' is the DataFrame containing the features and the binary label

# Extract the features and the target variable
X = technical_indicators_binary.drop('Label', axis=1)
y = technical_indicators_binary['Label']

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Create the RFE object with cross-validation and XGBoost as the classifier
rfecv = RFECV(estimator=xgb_classifier, step=1, cv=KFold(10), scoring='accuracy', n_jobs=-1)

# Fit the RFE object to the data
rfecv.fit(X, y)

# Print the optimal number of features
print("Optimal number of features : %d" % rfecv.n_features_)

# Get the selected features
selected_features = X.columns[rfecv.support_]
print("Selected features:", selected_features)

# Perform cross-validation with the selected features
cv_scores = cross_val_score(xgb_classifier, X[selected_features], y, cv=KFold(10))

# Print the mean cross-validation score
print("Mean cross-validation score: ", np.mean(cv_scores))

# Get the classification report using cross_val_predict
y_pred = cross_val_predict(xgb_classifier, X[selected_features], y, cv=KFold(10))
print(classification_report(y, y_pred))


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarnin

Optimal number of features : 12
Selected features: Index(['SMA7', 'SMA21', 'EMA7', 'EMA21', 'MACD', 'MACD_signal', 'RSI', 'ADX',
       'BB_upper', 'BB_middle', 'BB_lower', 'OBV'],
      dtype='object')
Mean cross-validation score:  0.5683900352618518
              precision    recall  f1-score   support

           0       0.49      0.45      0.47      1051
           1       0.62      0.66      0.64      1428

    accuracy                           0.57      2479
   macro avg       0.55      0.55      0.55      2479
weighted avg       0.56      0.57      0.57      2479

